# Formation of Turing patterns in 2D reaction-diffusion

In this case, we consider a simple 2D geometry comprised of two "compartments":
- surf - 2D surface
- edge - outer edges of the surface (1D)

We implement the Schnakenberg model as a simple system that exhibits Turing patterns in 2D. In this model, two species diffuse in a single compartment ("surf"), A and B. A is produced autocatalytically and inhibits the production of B. B degrades over time and positively regulates the production of A.

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
img_A = mpimg.imread('schnakenberg-diagram.png')
plt.imshow(img_A)
plt.axis('off')

Nondimensionalizing, with $\bar{A}=A/c_{ref}$ and $\bar{B}=B/c_{ref}$, the Schnakenberg model is typically written in the form:
$$
\partial_t{\bar{A}} = \gamma (a - \bar{A} + \bar{A}^2 \bar{B}) + \nabla^2 \bar{A}\\
\partial_t{\bar{B}} = \gamma (b - \bar{A}^2 \bar{B}) + d \nabla^2 \bar{B}
$$
where $a$ and $b$ are reaction constants, $\gamma$ is a scaling factor, and $d$ is the ratio of the two diffusion coefficients ($D_B/D_A$). One requirement for forming Turing patterns is that $D_B > D_A$, so $d$ must be greater than 1 (here, we set it equal to 20).

To define the system below, we recover the dimensional form of these equations (SMART requires a form with dimensions):
$$
\partial_t{A} = c_{ref} \gamma^* (a - \bar{A} + \bar{A}^2 \bar{B}) + D_A \nabla^2 A\\
\partial_t{B} = c_{ref} \gamma^* (b - \bar{A}^2 \bar{B}) + D_B \nabla^2 B
$$

where $\gamma^* = \gamma D_A / L^2$.

We solve these equations over a 1 by 1 square domain with no-flux boundary conditions.

We begin with the necessary imports:

In [ ]:
import dolfin as d
import sympy as sym
import numpy as np
import pathlib

from smart import config, common, mesh, model, visualization
from smart.units import unit
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
)
import logging

We will set the logging level to `INFO`. This will display some output during the simulation. If you want to get even more output you could set the logging level to `DEBUG`.

In [ ]:
logger = logging.getLogger("smart")
logger.setLevel(logging.INFO)

Futhermore, you could also save the logs to a file by attaching a file handler to the logger as follows.

```python
file_handler = logging.FileHandler("filename.log")
file_handler.setFormatter(logging.Formatter(smart.config.base_format))
logger.addHandler(file_handler)
```

We define the various units for use in the model. 

In [ ]:
# Aliases - base units
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
# Aliases - units used in model
D_unit = um**2 / sec
flux_unit = molecule / (um * sec)
surf_unit = molecule / um**2
edge_unit = molecule / um

## Generate model

In [ ]:
# define dimensions of domain
L = 1.0 # typical length scale for nondimensionalization
x_size = 1.0 * L
y_size = 1.0 * L

# =============================================================================================
# Compartments
# =============================================================================================
# name, topological dimensionality, length scale units, marker value
surf = Compartment("surf", 2, um, 1)
edge = Compartment("edge", 1, um, 3)
cc = CompartmentContainer()
cc.add([surf, edge])

# =============================================================================================
# Species
# =============================================================================================
# name, initial concentration, concentration units, diffusion, diffusion units, compartment
D_A = 0.0001 # diffusion coefficient of species A
d_ratio = 20.0 # ratio between diffusion coefficients
a_val = 0.1#0.14
b_val = 1#1.35
Ainit = a_val + b_val
Binit = b_val / (a_val + b_val)**2
# Ainit = f"{Ainit}*(1 + 0.1*sin(2*pi*x/.1))"
# Binit = f"{Binit}*(1 + 0.1*sin(2*pi*x/.1))"
A = Species("A", Ainit, surf_unit, D_A, D_unit, "surf") # activator
B = Species("B", Binit, surf_unit, d_ratio*D_A, D_unit, "surf") # inhibitor
sc = SpeciesContainer()
sc.add([A, B])

# =============================================================================================
# Parameters and Reactions
# =============================================================================================
# Describing the Schnakenberg model here
gStar = Parameter("gStar", 10000*D_A/L**2, 1/sec) # gStar = gamma*D_A/L^2
a = Parameter("a", a_val, dimensionless)
b = Parameter("b", b_val, dimensionless)
cref = Parameter("cref", 1.0, surf_unit) # to convert from dimensionless forms
L = Parameter("L", 1.0, um)

# Production of A
r1 = Reaction("r1", [], ["A"], 
              param_map={"a": "a", "gStar": "gStar", "cref":"cref"}, 
              eqn_f_str="cref*gStar*(a + (A/cref)**2 * (B/cref))", 
              species_map={"A": "A", "B": "B"})

# Degradation of A
r2 = Reaction("r2", ["A"], [], 
              param_map={"gStar": "gStar", "cref":"cref"}, 
              eqn_f_str="cref*gStar*A/cref", 
              species_map={"A": "A"})

# Production of B
r3 = Reaction("r3", [], ["B"], 
              param_map={"gStar":"gStar","b": "b", "cref":"cref"}, 
              eqn_f_str="cref*gStar*b")

# Degradation of B
r4 = Reaction("r4", ["B"], [], 
              param_map={"gStar": "gStar", "cref":"cref"}, 
              eqn_f_str="cref*gStar* (A/cref)**2 * (B/cref)", 
              species_map={"A": "A", "B": "B"})

pc =ParameterContainer()
pc.add([a, b, gStar, cref])
rc = ReactionContainer()
rc.add([r1, r2, r3, r4])

## Create/load in mesh

In SMART we have different levels of meshes. Here we create a UnitSquare mesh defined by

$$
\Omega = [0, 1] \times [0, 1] \subset \mathbb{R}^2
$$

which will serve as our parent mesh

For our two domains, we have two associated "child meshes":
- surf: in this case, all cells (triangles) belong to this mesh
- edge: 1D child mesh including all line elements along the edges of the domain

In [ ]:
# Create mesh
m = 100; n = int(x_size/y_size)*m
rect_mesh = d.RectangleMesh(d.Point(0.0, 0.0), d.Point(x_size, y_size), n, m)
mf2 = d.MeshFunction("size_t", rect_mesh, 2, 1)
mf1 = d.MeshFunction("size_t", rect_mesh, 1, 0)
for e in d.edges(rect_mesh):
    x_vals = np.zeros(2)
    y_vals = np.zeros(2)
    idx = 0
    for vertex in d.vertices(e):
        x_vals[idx] = vertex.point().array()[0]
        y_vals[idx] = vertex.point().array()[1]
        idx = idx+1
    if np.isclose(np.mean(x_vals), 0.) or np.isclose(np.mean(x_vals),x_size)\
        or np.isclose(np.mean(y_vals),0) or np.isclose(np.mean(y_vals),y_size):
        mf1[e] = 3
# Write mesh and meshfunctions to file
mesh_folder = pathlib.Path("rect_mesh")
mesh_folder.mkdir(exist_ok=True)
mesh_file = mesh_folder / "rect_mesh.h5"
hdf5 = d.HDF5File(rect_mesh.mpi_comm(), str(mesh_file.with_suffix(".h5")), "w")
hdf5.write(rect_mesh, "/mesh")
hdf5.write(mf2, "/mf2")
hdf5.write(mf1, "/mf1")
# For visualization of domains
d.File(str(mesh_file.with_stem(mesh_file.stem + "_mf2").with_suffix(".pvd"))) << mf2
d.File(str(mesh_file.with_stem(mesh_file.stem + "_mf1").with_suffix(".pvd"))) << mf1
parent_mesh = mesh.ParentMesh(
    mesh_filename=str(mesh_file),
    mesh_filetype="hdf5",
    name="parent_mesh",
)
visualization.plot_dolfin_mesh(rect_mesh, mf2)

## Initialize model and solver
Now we are ready to set up the model. First we load the default configurations and set the solver config.

In [ ]:
configCur = config.Config()
configCur.flags.update({"allow_unused_components": True})
modelCur = model.Model(pc, sc, cc, rc, configCur, parent_mesh)
configCur.solver.update(
    {
        "final_t": 50.0,
        "initial_dt": 0.1,
        "time_precision": 6,
        "use_snes": True,
        "print_assembly": False,
    }
)

We now initialize the model using the `initialize` function found in the `smart.model` module. We then perturb the initial conditions by adding white noise to the dolfin vectors associated with each species. Finally, we initialized the solver according to the config options we chose above.

In [ ]:
modelCur.initialize(initialize_solver=False)
# add white noise perturbation to initial conditions
for sp_str in ("A","B"):
    sp = modelCur.sc[sp_str]
    u = modelCur.cc[sp.compartment_name].u["u"]
    indices = sp.dof_map
    uvec    = u.vector()
    values  = uvec.get_local()
    values[indices] = np.multiply(values[indices], 
                                  np.random.normal(1, 0.01, len(indices)))
    uvec.set_local(values)
    uvec.apply("insert")
    nvec = modelCur.cc[sp.compartment_name].u["n"].vector()
    nvec.set_local(values)
    nvec.apply("insert")

modelCur.initialize_discrete_variational_problem_and_solver()

## Solve the system and write output data
Now, we are ready to start the solution process. We store the initial conditions to output files and then solve the system at each time step using the `monolithic_solve` function. Once we pass the final time chosen above, we exit the loop.

In [ ]:
# Write initial condition(s) to file
results = dict()
result_folder = pathlib.Path("resultsRect")
result_folder.mkdir(exist_ok=True)
for species_name, species in modelCur.sc.items:
    results[species_name] = d.XDMFFile(
        modelCur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
    )
    results[species_name].parameters["flush_output"] = True
    results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)

# Solve
while True:
    # Solve the system
    modelCur.monolithic_solve()
    # Save results for post processing
    for species_name, species in modelCur.sc.items:
        results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)
    # End if we've passed the final time
    if modelCur.t >= modelCur.final_t:
        break

Here, we plot our final solution, showing an example of a Turing pattern. The wavelength of this patterning is tunable by changing `D_ref` in the model definition above.

In [ ]:
%matplotlib inline
d.plot(modelCur.sc["B"].u["u"])
visualization.plot(modelCur.sc["B"].u["u"], show_edges=False)